In [3]:
!pip install wikipedia

In [4]:
!pip install arxiv

In [5]:
!pip install langchain-community

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [7]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
wiki.name

'wikipedia'

In [9]:
!pip install langchain

In [10]:
!pip install langchain_openai

In [11]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.6 kB)
Using cached faiss_cpu-1.8.0-cp312-cp312-macosx_11_0_arm64.whl (3.1 MB)


In [25]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(["https://msme.gov.in/","https://udyamregistration.gov.in/","https://en.wikipedia.org/wiki/Ministry_of_Micro_Small_and_Medium_Enterprises"])
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 20,chunk_overlap = 2).split_documents(docs)
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings(api_key="sk-proj-FfPGagp6wyt1vjEmk7IQT3BlbkFJI5a5XNrku0JtG4DlqIFm"))
retriever = vectordb.as_retriever()
retriever.name

In [26]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"sme_search",
                      "Search for information relevant to small and medium enterprises (SMEs). If you have any questions about SMEs, you can use this tool.")
retriever_tool.name

'sme_search'

In [27]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [28]:
tools = [retriever_tool,arxiv,wiki]
tools

[Tool(name='sme_search', description='Search for information relevant to small and medium enterprises (SMEs). If you have any questions about SMEs, you can use this tool.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1067507c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16a649400>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x106750c20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16a649400>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.S

In [29]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0,api_key="sk-proj-FfPGagp6wyt1vjEmk7IQT3BlbkFJI5a5XNrku0JtG4DlqIFm")

In [30]:
!pip install langchainhub

In [31]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [33]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [34]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Langsmith`


No good Wikipedia Search Result was foundI couldn't find any information about "Langsmith" on Wikipedia. Would you like me to search for information about Langsmith using a different source?

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': 'I couldn\'t find any information about "Langsmith" on Wikipedia. Would you like me to search for information about Langsmith using a different source?'}

In [35]:
agent_executor.invoke({"input":"Tell me about bangalore"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Bangalore`


Page: Bangalore
Summary: Bangalore ( BANG-gə-lor, -⁠LOR), officially Bengaluru (Kannada pronunciation: [ˈbeŋgɐɭuːɾu] ), is the capital and largest city of the southern Indian state of Karnataka. It haBangalore, officially known as Bengaluru, is the capital and largest city of the southern Indian state of Karnataka. It is a vibrant city known for its pleasant climate, green spaces, and thriving technology industry. If you would like more detailed information about Bangalore, feel free to ask!

> Finished chain.


{'input': 'Tell me about bangalore',
 'output': 'Bangalore, officially known as Bengaluru, is the capital and largest city of the southern Indian state of Karnataka. It is a vibrant city known for its pleasant climate, green spaces, and thriving technology industry. If you would like more detailed information about Bangalore, feel free to ask!'}

In [36]:
agent_executor.invoke({"input":"what is Small medium enterprises" })["output"]



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Small and medium-sized enterprises`


Page: Small and medium-sized enterprises
Summary: Small and medium-sized enterprises (SMEs) or small and medium-sized businesses (SMBs) are businesses whose personnel and revenue numbers fall below ceSmall and medium-sized enterprises (SMEs) or small and medium-sized businesses (SMBs) are businesses whose personnel and revenue numbers fall below certain limits. SMEs are commonly defined as businesses with fewer than 250 employees and annual revenue below a certain threshold. These enterprises play a crucial role in the economy by contributing to job creation, innovation, and economic growth.

> Finished chain.


'Small and medium-sized enterprises (SMEs) or small and medium-sized businesses (SMBs) are businesses whose personnel and revenue numbers fall below certain limits. SMEs are commonly defined as businesses with fewer than 250 employees and annual revenue below a certain threshold. These enterprises play a crucial role in the economy by contributing to job creation, innovation, and economic growth.'

In [ ]:
agent_executor.invoke({"input":"what is the eligibility for applying for Small medium enterprises" })["output"]



> Entering new AgentExecutor chain...

Invoking: `sme_search` with `{'query': 'eligibility for applying for Small Medium Enterprises'}`


Medium Enterprises

Medium Enterprises

Medium Enterprises

entitled to do MSME
Invoking: `sme_search` with `{'query': 'eligibility criteria for applying for Small and Medium Enterprises'}`
responded: It seems like there was an issue with the search results. Let me try to refine the search query to provide you with accurate information on the eligibility criteria for applying for Small and Medium Enterprises (SMEs).

Medium Enterprises

Medium Enterprises

Medium Enterprises

MSME registration
Invoking: `sme_search` with `{'query': 'eligibility criteria for Small and Medium Enterprises'}`
responded: It appears that the search results are still not providing the specific eligibility criteria for applying for Small and Medium Enterprises (SMEs). Let me try another approach to gather the required information for you.

Medium Enterprises

Medium Enterp

'I apologize for the inconvenience. The search results are still not providing the specific eligibility criteria for applying for Small and Medium Enterprises (SMEs). \n\nIn general, the eligibility criteria for SMEs may vary depending on the country or region. However, some common eligibility requirements for SMEs may include:\n\n1. Business Size: Typically, SMEs are defined based on the number of employees and annual revenue. The specific thresholds for small and medium enterprises may vary by industry and location.\n\n2. Legal Status: The business should be registered as a legal entity, such as a sole proprietorship, partnership, limited liability company, etc.\n\n3. Compliance: The business should comply with all legal and regulatory requirements, including tax obligations, licenses, permits, and certifications.\n\n4. Business Plan: Having a well-defined business plan outlining the objectives, target market, products or services, and financial projections may be required.\n\n5. Fin